In [23]:
import pandas as pd
import nltk 
import numpy as np
import re
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import twitter_samples
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, hamming_loss

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def limpiar_tweet(tweet):
    
    stemmer = PorterStemmer()  # Inicializar PorterStemmer
    stop_words = nltk.corpus.stopwords.words("spanish")  # Cargar stopwords en inglés
    bad_chars = ["$","USD","€","£","¥","%","&","(",")",",",".","?","¡","¿",'"',"'"]
    tweet = ''.join(i for i in tweet if not i in bad_chars)  # Eliminar caracteres no deseados
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet) # Eliminar url
    tweet = re.sub(r'@\w+', '', tweet)  # Eliminar menciones
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)  # Eliminar caracteres especiales
    tweet_tokens = tweet.lower().split()  # Tokenizar el tweet
    tweet_lista = []
    for word in tweet_tokens:
        if word not in stop_words:
            tweet_lista.append(word)
    return tweet_lista

In [43]:
df_train = pd.read_csv("toxic_train.csv", delimiter=";")
df_train['tweet_limpio'] = df_train['text'].apply(limpiar_tweet)
df_train['tweet_limpio'] = df_train['tweet_limpio'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
df_train_limpio = df_train.drop(columns=['id','text'])
df_train_limpio.head(3)

,HS,TR,AG,tweet_limpio
0,0,0,0,amigos saime nombre walter perez v wperez cita...
1,0,0,0,preferan enfrentar peligros regresar hogar ser...
2,0,0,0,siiii jajjajajajjajajaja iba ensear vivs lejos...


In [54]:
df_test = pd.read_csv("toxic_test.csv", delimiter=";", encoding='latin-1')
df_test['tweet_limpio'] = df_test['text'].apply(limpiar_tweet)
df_test['tweet_limpio'] = df_test['tweet_limpio'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
df_test_limpio = df_test.drop(columns=['text'])
df_test_limpio.head(3)

,id,tweet_limpio
0,20929,indocumentado
1,22807,cllate perra deje ventana balcn apto abierta j...
2,24595,echo puta madre ensearte ahora vamos pocos tom...


In [49]:

# Preparar datos
X = df_train_limpio['tweet_limpio']  # tus tweets tokenizados
y = df_train_limpio[['HS', 'TR', 'AG']]  # las 3 etiquetas

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [50]:
# Vectorización
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [51]:
modelo = MultiOutputClassifier(
    LogisticRegression(max_iter=1000, class_weight='balanced', C=1.0)
)

In [52]:
# Entrenar
print("\nEntrenando modelo...")
modelo.fit(X_train_vec, y_train)


Entrenando modelo...


MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   max_iter=1000))

In [53]:
# Predecir
y_pred = modelo.predict(X_test_vec)

# Evaluar
print("Hamming Loss:", hamming_loss(y_test, y_pred))
for i, col in enumerate(['HS', 'TR', 'AG']):
    print(f"\n{col}:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Hamming Loss: 0.16810758885686838

HS:
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       200
           1       0.79      0.73      0.76       147

    accuracy                           0.80       347
   macro avg       0.80      0.79      0.79       347
weighted avg       0.80      0.80      0.80       347


TR:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       240
           1       0.79      0.83      0.81       107

    accuracy                           0.88       347
   macro avg       0.86      0.87      0.86       347
weighted avg       0.88      0.88      0.88       347


AG:
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       205
           1       0.81      0.70      0.75       142

    accuracy                           0.81       347
   macro avg       0.81      0.80      0.80       347
weighted avg       0.81  

In [56]:
X_test_vec = vectorizer.transform(df_test_limpio['tweet_limpio'])

In [ ]:
y_pred = modelo.predict(X_test_vec)


In [59]:
submission = pd.DataFrame({
    'id': df_test_limpio['id'],
    'HS': y_pred[:, 0],
    'TR': y_pred[:, 1],
    'AG': y_pred[:, 2]
})

In [61]:
submission.to_csv("submission.csv", index=False)